# Coursera Capstone Project Notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Capstone Project - Week 1
### 1. Introduction/Business Problem

A young professional, new to London, United Kingdom, is looking for a neighborhood to move to. They are looking at 3 particular areas in North London: Holland Park and Notting Hill, Angel and Islington, and Shoreditch and Aldgate (which could be grouped into their respective Boroughs).
They are looking for an area with good rent, and lots of amenities such as cafes, gyms, bars, etc, nearby. As a newcomer to London, they are hoping that they will be surrounded by residents aroudn their age, mid 20s. They are hoping also for good transportation links to their new job near London Bridge, as well as to central London.

This will be useful to compare different neighborhoods with one another to see which one best fits with the users' specfic interests. With additional analysis on local schools, it would also be beneficial for families looking for neighborhoods to settle into.

### 2. Data needed

In order to determine the best neighborhood for our user, we will be using Foursquare location data of London to determine the venues nearby these three areas. We can group the venues by type, such as cafes, gyms, grocery stores, and subcategories such as type of cuisine for resturants or brunch offerings, and determine how many of each there are. Further analysis can be done to determine popularity of venues during certain times, such as Thursday to Saturday evenings which are known to return a high turnout of young people at particular venues such as bars and clubs.

Transportation data can also be used to determine the number of major bus or tube routes that pass through the neighborhood to get to London Bridge and/or Central London, in order to determine how well connected the neighorhood is.
